In [35]:
import datasets
from datasets import load_dataset
import tokenizers
import tqdm

import os
import torch

import torch.nn as nn
from torch.nn import functional as F

In [36]:
torch.manual_seed(1339)

In [1]:
tinystories_ds = load_dataset("roneneldan/TinyStories")
tinystories_sp_ds = load_dataset("robrenaud/multilingual_tinystories")

Repo card metadata block was not found. Setting CardData to empty.


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Resolving data files:   0%|          | 0/54 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [10]:
with open('train.txt', 'w') as full_stories_output :
    for story in tqdm.tqdm(tinystories_sp_ds['train']['story']):
        full_stories_output.write(story)
        
    for story in tqdm.tqdm(tinystories_ds['train']['text']):
        full_stories_output.write(story)

100%|████████████████████████████████████████████████████████████████████| 2119719/2119719 [00:01<00:00, 1542390.21it/s]


In [12]:
tokenizer = tokenizers.ByteLevelBPETokenizer()

tokenizer.train(files=['train.txt'], vocab_size=2**13, min_frequency=2)

tokenizer.save_model('.', 'tiny-stories-bpe')

['./tiny-stories-bpe-vocab.json', './tiny-stories-bpe-merges.txt']

In [17]:
from itertools import chain

In [18]:
stories = chain(tinystories_ds['train']['text'], tinystories_sp_ds['train']['story'])

In [ ]:
if not os.path.isdir('tokenized'):
    os.mkdir('tokenized')

In [24]:
output_buf = []
num_outputs = 0
for story in tqdm.tqdm(stories):
    encoded = torch.tensor(tokenizer.encode(story).ids, dtype=torch.short)
    output_buf.append(encoded)
    if len(output_buf) > 500_000:
        torch.save(output_buf, f'tokenized/tokenized-{num_outputs}.pt')
        num_outputs += 1
        output_buf = []
if output_buf:
    torch.save(output_buf, f'tokenized/tokenized-{num_outputs}.pt')
    num_outputs += 1
    output_buf = []

7423861it [1:03:03, 1962.41it/s]


## Train

In [25]:
#----- imports --------
import tqdm
import torch
# import wandb
import os
import tokenizers


device= 'cuda' if torch.cuda.is_available() else 'cpu'
torch.set_default_device(device)
assert device == 'cuda', "This notebook is not optimized for CPU"

config = {
    "learning_rate": 1e-3,
    "eval_interval": 300,
    "max_iters": 60000, 
    "H": 32, # hidden dimension size
    "B": 64,
    "T": 256, # Sequence length
    "C": 256,
    "feedforward_factor": 3,
    "n_heads": 8,
    "dropout": 0.0,
    "l2_penalty": 0.0,
    "n_layers": 12,
    "tokenizer_vocab_size": 2**13,
    # "git_hash": os.popen("git rev-parse HEAD").read().strip()
}

# initial
for k,v in config.items():
    locals ()[k] = v

In [26]:
def load_sharded_story(shard_no):
    return torch.load(f'tokenized/tokenized-{shard_no}.pt')

In [27]:
# load the tokenized stories in parallel using threads
# this is faster than loading them sequentially
from concurrent.futures import ThreadPoolExecutor
with ThreadPoolExecutor() as pool:
    stories = list(tqdm.tqdm(pool.map(load_sharded_story, range(1)), total=1))

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.38s/it]


In [28]:
all_stories = []
for story in stories:
    all_stories.extend(story)

In [29]:
print("length of dataset in stories: ", len(all_stories))
print("length of stories in tokens", sum(len(story) for story in all_stories))

length of dataset in stories:  500001
length of stories in tokens 114224513


In [30]:
num_stories_to_check = 1_000_000
num_long = sum(len(story) > T for story in all_stories[:num_stories_to_check])
print(
    f"# stories longer than {T} : {num_long} out of {num_stories_to_check}, {num_long/num_stories_to_check:.2%}")

# stories longer than 256 : 105105 out of 1000000, 10.51%


In [31]:
def encode(text):
    return tokenizer.encode(text).ids
def decode(encoded_text):
    return tokenizer.decode(encoded_text)

from tqdm import tqdm

def batch_encode(text, batch_size):
    tokens = []
    for i in tqdm(range(0, len(text), batch_size)):
        tokens.extend(encode(text[i:i+batch_size]))
    return tokens


hello_encoded = encode("Hola")
print(hello_encoded)
print(decode(hello_encoded))
vocab_size = tokenizer.get_vocab_size()
print("vocab size: ", vocab_size)
print('first story decoded: ', decode(all_stories[0].tolist()))
PADDING_TOKEN_IDX= encode(" ")[0]

[705]
Hola
vocab size:  8192
first story decoded:  Once upon a time, there was a little car named Beep. Beep loved to go fast and play in the sun. Beep was a healthy car because he always had good fuel. Good fuel made Beep happy and strong.

One day, Beep was driving in the park when he saw a big tree. The tree had many leaves that were falling. Beep liked how the leaves fall and wanted to play with them. Beep drove under the tree and watched the leaves fall on him. He laughed and beeped his horn.

Beep played with the falling leaves all day. When it was time to go home, Beep knew he needed more fuel. He went to the fuel place and got more healthy fuel. Now, Beep was ready to go fast and play again the next day. And Beep lived happily ever after.


In [32]:
n = int(0.9*len(all_stories))

train_data = all_stories[:n]
val_data = all_stories[n:]

In [33]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(0, len(data), (B,)) 
    # ix = [i for i in range(B)]
    
    x = torch.full((B, T), PADDING_TOKEN_IDX, dtype=torch.long)
    y = torch.full((B, T), PADDING_TOKEN_IDX, dtype=torch.long)

    for sequence_index, random_story_index in enumerate(ix):
        story = data[random_story_index].long()[:T - 1]
        x[sequence_index][1: story.shape[0]+1] = story
        y[sequence_index][: story.shape[0]] = story

    return x, y

xb, yb = get_batch('train')

print(xb[0])
print(yb[0])

tensor([ 220,  670,  708,  256,  601,   11,  605,  348,  256, 2743, 2596,  700,
          13,  626,  348, 1022,  309, 3945,   11,  455,  398, 2611,  309, 1160,
         300,  291,   13,  480,  602,  707, 1428,  292, 2596,  700,  549, 5595,
         309, 7032,  301,  794,   13,  428, 2433,  715,  401, 1350,  309, 4406,
         291,  292, 2596,  700,  483,  256, 1044,   13,  375, 2596,  700,  348,
         455,  616,  301,  749, 2970, 1320,  398, 1676,   13,  626,  846,  309,
         292,  707,  633,  521,  398,  528,  256, 3286,  367,  421, 1761, 6131,
          13,  313,  198,  559,  707,  309,  292, 2596,  700, 2207,  790, 2043,
         521,   13,  428,  914, 6042,  533,  488,  401, 2596,  700, 1141,  515,
         309, 3040,  398, 1316,  398,  348, 1933,   13,  375, 2596,  700,  348,
        1221,  616,  301,  936,  401,   13,  375,  707, 1221, 1425,  292, 2596,
         700,  256,  615, 1044,  309,  462,   11, 1684,  933, 4547, 1494, 2280,
         527,  495,    0, 1151,  933,  1

In [37]:
class Head(nn.Module):
    '''One Head of self-attention'''
    def __init__(self, H):
        super().__init__()
        self.query = nn.Linear(C, H, bias=False)
        self.key = nn.Linear(C, H, bias=False)
        self.value = nn.Linear(C, H, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(T, T)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        query_vectors = self.query(x)
        key_vectors = self.key(x)


        # Attention masking(so we can't look into the past):
        tril = self.tril
        wei = torch.zeros(T, T) 
        wei = wei.masked_fill(tril == 0, float('-inf')) # set the upper triangular to -inf

        # multiply the two to get the attention weights
        attention_pattern = query_vectors @ key_vectors.transpose(-2, -1) # T, T
        attention_pattern = attention_pattern / (H ** 0.5) # scale the attention pattern for numerical stability
        attention_weights = F.softmax(attention_pattern + wei, dim=-1) # T, T (the row dimension is the query)
        attention_weights = self.dropout(attention_weights)

        value_vectors = self.value(x) # the direction we should go in the embedding space for each token (ie more blue) T, H

        # apply the attention weights to the value vectors
        context = attention_weights @ value_vectors # T, H

        # project back into original space from value space
        return context

x = torch.randn(B,T,C)
head = Head(H)

In [38]:
class MultiHeadAttention(nn.Module):
    '''Multiple heads of self-attention'''
    def __init__(self, H, C, n_heads): # H is head embedding space size, n_heads is number of heads
        super().__init__()
        self.heads = nn.ModuleList([Head(H) for _ in range(n_heads)])
        self.combine_heads = nn.Linear(H*n_heads, C)
        self.dropout = nn.Dropout(dropout)

    def forward(self,x):
        x = torch.cat([head(x) for head in self.heads], dim=-1)
        x = self.combine_heads(x)  # T, C
        return self.dropout(x)

In [39]:
head = MultiHeadAttention(H, C, n_heads)
head.heads[0].forward(x).shape

torch.Size([64, 256, 32])

In [40]:
class FeedForward(nn.Module):
    '''Feed-forward neural network'''
    def __init__(self, C):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(C, C * feedforward_factor),
            nn.ReLU(),
            nn.Linear(C * feedforward_factor, C),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

In [41]:
class LayerNorm(nn.Module):
    '''Layer normalization'''
    def __init__(self, C, use_affine=True):
        super().__init__()
        self.gamma = nn.Parameter(torch.ones(C)) if use_affine else None
        self.beta = nn.Parameter(torch.zeros(C)) if use_affine else None
        
    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        if self.gamma is not None and self.beta is not None:
            return self.gamma * (x - mean) / (std + 1e-6) + self.beta
        else:
            return (x - mean) / (std + 1e-6)

In [42]:
class Block(nn.Module):
    '''Transformer block'''
    def __init__(self, H, C, n_heads):
        super().__init__()
        self.attention = MultiHeadAttention(H, C, n_heads)
        self.ff = FeedForward(C)
        self.norm1 = LayerNorm(C, use_affine=True)
        self.norm2 = LayerNorm(C, use_affine=True)

    def forward(self, x):
        x = x + self.attention(self.norm1(x))
        x = x + self.ff(self.norm2(x))
        return x

In [44]:
class GPT(nn.Module):
    
    def __init__(self, n_layers):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, C)
        self.position_embedding_table = nn.Embedding(T, C)
        self.lm_head = nn.Linear(C, vocab_size)
        self.layers = nn.ModuleList([Block(H, C, n_heads) for _ in range(n_layers)])
        self.block = nn.ModuleList([Block(H, C, n_heads)])
    
    def forward(self, idx, targets=None):
        B, T = idx.shape
        token_emb = self.token_embedding_table(idx) # batch_dim, sequence_dim, embedding_dim
        pos_emb = self.position_embedding_table(torch.arange(T))
        x = token_emb + pos_emb # token identities and positions contained

        for layer in self.layers:
            x = layer(x)

        logits = self.lm_head(x) # batch_dim, sequence_dim, vocab_size

        batch_dim, sequence_dim, embedding_dim = logits.size()

        if targets is None:
            return logits, None
        else:
            logits_loss_view = logits.view(-1, vocab_size) 
            targets_loss_view = targets.view(-1)
            loss = F.cross_entropy(logits_loss_view, targets_loss_view)
            return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx[:,-T:])
            # get the predictions of the last token
            last_token_logits = logits[:, -1, :] # all batches, last token, all probabilities
            # softmax to get probabilities
            probabilities = F.softmax(last_token_logits, dim=-1)
            # sample from the probabilities
            next_token = torch.multinomial(probabilities, num_samples=1)
            # add the new token to the idx tensor
            idx = torch.cat((idx, next_token), dim=1)
        return idx
    def prompt_model(self, prompt, max_new_tokens, temperature=0.5):
        autoregressive_seq = encode(prompt)
        for _ in range(max_new_tokens):
            prediction_index = len(autoregressive_seq)-1

            model_input = torch.tensor(autoregressive_seq)
            
            while model_input.shape[0] < T:
                pad_token = torch.tensor(encode("\n"))
                model_input = torch.cat((model_input, pad_token), dim=0)

            model_input
            model_input = model_input.unsqueeze(0)

            logits, loss = model(model_input)
            prediction_token = logits[:, prediction_index, :] / temperature
            probabilities = F.softmax(prediction_token, dim=-1)
            next_token = torch.multinomial(probabilities, num_samples=1)
            next_token = next_token.item()

            autoregressive_seq.append(next_token)
        # get the autoregressive sequence
        return decode(autoregressive_seq)

In [45]:
model = GPT(n_layers)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

test_idx = torch.zeros(1, T).long()
model.forward(idx=test_idx)
decode(model.generate(idx=test_idx, max_new_tokens=100)[0].tolist())

torch.Size([64, 256, 8192])
tensor(9.6883, device='cuda:0', grad_fn=<NllLossBackward0>)


'!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! peluche witch dor Bellafiaess crocodile nobodyJugaron forgot knife librarUno piernas grabcepAllœThat why surprise disfraces cansaron.. esté buenaSor pud cla thought mujer crawl Voy lento nutslass sighed actTom alguien sharing smile últ disgusting está goatlo malosCarlos brazBilly� flour ayudar natura dependable olía dinosau Todorimero held empu sing levantarcito aburrido vive held ríe llevarlo firstTommy rota fut helicó restaurantel further tomaron Jack truenoluffy balloon gusta hands rota steal díauestabalGrapuedo antscí madto vamos pequeñaenryará puddle'

In [46]:
# get the number of parameters in the model
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("number of parameters in the model: ", count_parameters(model))

number of parameters in the model:  12817664


In [47]:
idx = torch.zeros(1, 1).long()
idx[:,-T:]

tensor([[0]], device='cuda:0')

In [48]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [49]:
eval_iters = 10
eval_interval = 300

@torch.no_grad()
def estimate_loss(is_last=False):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        real_iters = eval_iters
        if is_last and split == 'val':  # increase last eval to mitigate noise
            real_iters *= 10 
        losses = torch.zeros(real_iters)
        for k in range(real_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean() / chars_per_token
    model.train()
    return out

In [ ]:
import tqdm
num_params = sum([p.numel() for p in model.parameters()])

dump_model_interval = 1000
chars_per_token=3.9

for steps in tqdm.tqdm(range(max_iters)):
    xb, yb = get_batch('train')
    # loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)

    loss.backward()
    optimizer.step()
    if steps % eval_interval == 0:
        losses = estimate_loss()
        # wandb.log({"train": losses['train'].item(), "val": losses['val'].item(), "l2":l2})
        print({"train": losses['train'].item(), "val": losses['val'].item()})
    if steps % dump_model_interval == 0 and steps > 0:
        model_no = steps // dump_model_interval
        torch.save(model.state_dict(), f'tiny-stories-model-{model_no}.pt')

losses = estimate_loss(is_last=True)

  0%|                                                                               | 2/60000 [00:01<8:26:31,  1.97it/s]

{'train': 2.2240588665008545, 'val': 2.228090286254883}


  1%|▍                                                                            | 302/60000 [00:42<4:44:10,  3.50it/s]

{'train': 0.6910275816917419, 'val': 0.7089463472366333}


  1%|▊                                                                            | 602/60000 [01:22<4:47:52,  3.44it/s]

...

 60%|███████████████████████████████████████████▍                             | 35702/60000 [1:19:54<1:48:27,  3.73it/s]

{'train': 0.30031031370162964, 'val': 0.31947100162506104}


 60%|███████████████████████████████████████████▊                             | 36001/60000 [1:20:33<2:35:26,  2.57it/s]

{'train': 0.2992815673351288, 'val': 0.3160003125667572}


 60%|█████████████████████████████████████████████                              | 36021/60000 [1:20:35<54:21,  7.35it/s]

In [ ]:
estimate_loss()

In [ ]:
torch.save(model.state_dict(), 'tiny-stories-model.pt')

In [ ]:
test_idx = torch.zeros(1, T).long() * 198
print(decode(
    model.generate(idx=test_idx, max_new_tokens=C)[0].tolist()
)[T:])